In [1]:
import pandas as pd
import numpy as np
import json
import requests
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

#Need to use a delay between page scrapes in order to limit getting blocked by Yelp
from time import sleep

In [7]:
#ENTER SEARCH TERMS BELOW:
cuisine_type = "food"
location = "Argentina"

#Generate URL based on search terms
base_url = "https://www.yelp.com"
search_url = f"{base_url}/search?find_desc={cuisine_type}&find_loc={location}"

#Or manually set search_url by copying directly from Yelp Page if desired
#search_url = "https://www.yelp.com/search?find_desc=burger&find_loc=Albany%2C%20NY"

In [2]:
star_container_class = "attribute__09f24__3znwq display--inline-block__09f24__3L1EB margin-r1__09f24__BCulR border-color--default__09f24__1eOdn"
next_page_class = "next-link navigation-button__09f24__3F7Pt css-166la90"
search_result_class = "container__09f24__21w3G hoverable__09f24__2nTf3 margin-t3__09f24__5bM2Z margin-b3__09f24__1DQ9x padding-t3__09f24__-R_5x padding-r3__09f24__1pBFG padding-b3__09f24__1vW6j padding-l3__09f24__1yCJf border--top__09f24__8W8ca border--right__09f24__1u7Gt border--bottom__09f24__xdij8 border--left__09f24__rwKIa border-color--default__09f24__1eOdn"

In [50]:
# next_page_url = search_url
business_list = []

def scrape(location, next_page_url):
    page_counter = 1
    #Run continuously until there is no longer a "next page" url found.
#     while next_page_url:
    #Request HTML page and load into Beautiful Soup object
    request = requests.get(next_page_url)
    soup = BeautifulSoup(request.content,'html.parser')
    #Find search results container on page.
    search_results = soup.findAll(class_=search_result_class)
    print(f"Page {page_counter}, {len(search_results)} results {next_page_url}")
    #Loop through search results and store information for each business
    for search_result in search_results:
        business_info = {}
        if len(location) == 2:
            location = "AMERICA"
        business_info['location'] = location
        try:
            business_name_url = search_result.findAll('a', href=True)[1]
            business_info['url'] = f"https://www.yelp.com{business_name_url['href']}"
            business_info['name'] = business_name_url['name']
            business_info['biz_id'] = business_name_url['href'].split('/biz/')[1].split('?')[0]
        except:
            continue

        try:
            business_info['address'] = search_result.find('address').text
        except:
            pass

        try:
            business_info['category'] = [category.text for category in search_result.findAll("a",attrs={"role":"link"})]
        except:
            pass

        try:
            business_info['star_rating'] = float(re.findall(r"[-+]?\d*\.\d+|\d+",
                                                      search_result.find(
                                                          class_=star_container_class).find('div')['aria-label'] )[0])
        except:
            pass

        #Append business information for each search result to a list containing all businesses.
        if business_info:
            business_list.append(business_info)

        #Set url for next page. If not found, break out of loop.
#         if soup.find(class_=next_page_class):
#             next_page_url = base_url + soup.find(class_=next_page_class)['href']
#             page_counter+=1
#         else:
#             break

#         Random delay between 2 and 20 seconds to prevent getting blocked
#         sleep(3)

# scrape(next_page_url)
# print(len(business_list), "businesses scraped")

In [55]:
location_url = f"{base_url}/locations"
location_class = "state"

request = requests.get(location_url)
soup = BeautifulSoup(request.content,'html.parser')
#Find search results container on page.
search_results = soup.findAll(class_=location_class)

location_list = []
for result in search_results:
    location_list.extend(re.findall(r"\>([A-Z]+\s?[A-Z]*)\<",str(result.find('div'))))
    
print(location_list)

['ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'BELGIUM', 'BRAZIL', 'CANADA', 'CHILE', 'CZECH REPUBLIC', 'DENMARK', 'FINLAND', 'FRANCE', 'GERMANY', 'HONG KONG', 'ITALY', 'JAPAN', 'MALAYSIA', 'MEXICO', 'NEW ZEALAND', 'NORWAY', 'PHILIPPINES', 'POLAND', 'PORTUGAL', 'SINGAPORE', 'SPAIN', 'SWEDEN', 'SWITZERLAND', 'TAIWAN', 'THE NETHERLANDS', 'TURKEY', 'UNITED KINGDOM', 'UNITED KINGDOM', 'AZ', 'CA', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KY', 'LA', 'MA', 'MD', 'MI', 'MN', 'MO', 'NC', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OR', 'PA', 'RI', 'SC', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI']


In [57]:
count = 0
for element in location_list:
    cuisine_type = "food"
    location = element

    #Generate URL based on search terms
    base_url = "https://www.yelp.com"
    search_url = f"{base_url}/search?find_desc={cuisine_type}&find_loc={location}"
    
    scrape(location, search_url)
#     count = count + 1
#     if count == 2:
#         break
    sleep(5)

Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=ARGENTINA
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=AUSTRALIA
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=AUSTRIA
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=BELGIUM
Page 1, 9 results https://www.yelp.com/search?find_desc=food&find_loc=BRAZIL
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=CANADA
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=CHILE
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=CZECH REPUBLIC
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=DENMARK
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=FINLAND
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=FRANCE
Page 1, 10 results https://www.yelp.com/search?find_desc=food&find_loc=GERMANY
Page 1, 10 results https://www.yelp.com/search?

In [ ]:
print(business_list)

In [47]:
df = pd.DataFrame(business_list)
df.to_csv('yelp.csv') 